In [1]:
from coal_mapper import *
import os

### Initial Functions – Cleaning eGrid Data

In [2]:
def findYear(str):
    
    '''Finds the eGrid Data year within the file name passed to the function.
    Used for plant labeling purposes – seperating active from decomissioned plants'''
    str = str.lower()
    for i in range(0,len(str)):
        if (str[i:i+5] == "egrid"):
            return str[i+5:i+9]

In [3]:
def makeYearintoSheetName(str):
    '''Due to the nature of the EIA eGrid datasets, each .xlsx file has many sheet names.
    This function ensures the PLNT sheet is selected and read in'''
    year = findYear(str)
    return "PLNT"+year[2:4]

In [4]:
def findHeader(df):
    '''Because the eGrid datasheets are mnot standardized from year to year, this function is a necessity.
    Finds the row of data labels, allowing us to make it the top of our pd.Dataframe'''
    df2 = df
    test = df2[df2.columns[1]]
    for i in range(0, 10):
        if test[i] == "AK":
            break
        nmu = i
    return nmu

In [5]:
def isDecom(id):
    if id == "2020":
        return 0
    else:
        return 1

comment outlined sections of code to change to switch plant fuel type in the dataset; this study focuses only on coal

In [6]:
def cleanEIAegrid(str):
    '''Reads in and cleans an eGrid .xlsx
    Cleans it for use with the Kepler Mapper'''
    
    df_n = pd.read_excel(str, makeYearintoSheetName(str), header = 1)

    if (int(findYear(str)) < 2014):
        df = pd.read_excel(str, makeYearintoSheetName(str), header = findHeader(df_n)+2)
    else:
        df = df_n

    eyear = findYear(str)
    
    
#### comment out the below code to create dataset of all plants, not just coal. ####
#### edit code below to create a dataset of any powerplant fuel category. ####
    df = df[df['PLFUELCT'] == 'COAL']
    df2 = df[df['COALFLAG'] == 1]
    if (df2.size <2):
        df2 = df[df['COALFLAG'] == 'Yes']
    df = df2
#### end edits section ####

    df['Year'] = eyear
    df = df[['Year', 'ORISPL', 'PSTATABB', 'LAT', 'LON', 'PNAME', 'FIPSST', 'FIPSCNTY',  'CAPFAC', 'PLNGENAN', 'PLNOXAN', 'PLSO2AN', 'PLCO2AN', 'PLFUELCT']]
    df = df.assign(label=eyear + ": " + df['PNAME'] + ", " + df['PSTATABB'])
    df.reset_index(drop=True, inplace=True)
    df = df.dropna()

    return df

In [7]:
def makeEIADataSet(list):
    '''Takes a list of EIA eGRID datasets as the input. Creates a combined coal_mapper dataset out of each excel file.
    
    EX. list = os.listdir("egrid 2005-2020") where "egrid 2005-2020" is a folder containing all EIA eGrid datasets with a sheet named PLNT05 (or any other year suffix) from the subset of years.'''

    df = pd.DataFrame()

    for i in list:
        df = pd.concat([cleanEIAegrid(i), df], axis=0)

    dfCOPY = df
    dfCOPY = dfCOPY.sort_values(['ORISPL', 'Year'], ascending=[True, True])
    dfCOPY = dfCOPY.drop_duplicates(subset = "ORISPL", keep = 'last')
    dfCOPY = dfCOPY.reset_index()
    dfCOPY = dfCOPY.iloc[: , 1:]
   
    dfCOPY['decom'] = dfCOPY['Year'].apply(isDecom)
    dfCOPY['CarbonIntensity'] = dfCOPY['PLCO2AN'] / dfCOPY['PLNGENAN']

    dfCOPY = dfCOPY[dfCOPY.CAPFAC > 0]

    return dfCOPY

### An Example of what it looks to refrence the folder of eGrid data used to create the dataset

In [8]:
list = os.listdir("egrid 2005-2020")
print(list)

['eGRID2012_Data.xlsx', '.DS_Store', 'egrid2019_data.xlsx', 'egrid2020_data.xlsx', 'eGRID2010_Data.xls', 'egrid2018_data_v2.xlsx', 'eGRID2009_data.xls', 'egrid2016_data.xlsx', 'eGRID2014_Data_v2.xlsx', 'eGRID2005_plant.xls']


In [9]:
list.remove('.DS_Store')
print(list)

['eGRID2012_Data.xlsx', 'egrid2019_data.xlsx', 'egrid2020_data.xlsx', 'eGRID2010_Data.xls', 'egrid2018_data_v2.xlsx', 'eGRID2009_data.xls', 'egrid2016_data.xlsx', 'eGRID2014_Data_v2.xlsx', 'eGRID2005_plant.xls']


# Dataset Creation

In [10]:
whole = makeEIADataSet(list)
whole.head()

,Year,ORISPL,PSTATABB,LAT,LON,PNAME,FIPSST,FIPSCNTY,CAPFAC,PLNGENAN,PLNOXAN,PLSO2AN,PLCO2AN,PLFUELCT,label,decom,CarbonIntensity
0,2019,1.0,AK,55.339722,-160.497222,Sand Point,2,13.0,0.10788,3780.0,46.290,4.195,2362.834,OIL,"2019: Sand Point, AK",1,0.625088
1,2020,3.0,AL,31.006900,-88.010300,Barry,1,97.0,0.33150,10499146.0,1656.340,1283.600,6846726.509,COAL,"2020: Barry, AL",0,0.652122
2,2020,7.0,AL,34.012800,-85.970800,Gadsden,1,55.0,0.04172,50435.0,43.278,0.232,40707.358,GAS,"2020: Gadsden, AL",0,0.807125
3,2019,8.0,AL,33.644344,-87.196486,Gorgas,1,127.0,0.03954,490760.0,490.525,142.687,598479.725,COAL,"2019: Gorgas, AL",1,1.219496
4,2020,9.0,TX,31.756900,-106.375000,Copper Station,48,141.0,0.04452,33888.0,99.154,0.194,39306.020,GAS,"2020: Copper Station, TX",0,1.159880


In [15]:
whole.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4041 entries, 0 to 4613
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             4041 non-null   object 
 1   ORISPL           4041 non-null   float64
 2   PSTATABB         4041 non-null   object 
 3   LAT              4041 non-null   float64
 4   LON              4041 non-null   float64
 5   PNAME            4041 non-null   object 
 6   FIPSST           4041 non-null   int64  
 7   FIPSCNTY         4041 non-null   float64
 8   CAPFAC           4041 non-null   float64
 9   PLNGENAN         4041 non-null   float64
 10  PLNOXAN          4041 non-null   float64
 11  PLSO2AN          4041 non-null   float64
 12  PLCO2AN          4041 non-null   float64
 13  PLFUELCT         4041 non-null   object 
 14  label            4041 non-null   object 
 15  decom            4041 non-null   int64  
 16  CarbonIntensity  4041 non-null   float64
 17  fuel          

### example sorting a dataset including all powerplants into fuel categories that can be viewed in a mapper

In [ ]:
def fuelType(id):
    if id == 'COAL':
        return 0
    elif id == "GAS":
        return 1
    elif id == "OIL":
        return 2
    else:
        return 4

In [14]:
whole['fuel'] = whole['PLFUELCT'].apply(fuelType)

In [ ]:
#map with 2 lenses

from sklearn import ensemble, cluster

df = whole
df = df[df['decom']==0]
df2 = StandardScaler().fit_transform(df[['Year','PLCO2AN', 'PLSO2AN', 'PLNOXAN', 'CarbonIntensity']])
df3 = df[['PLCO2AN', 'PLSO2AN', 'PLNOXAN', 'CarbonIntensity', 'fuel']]

colors = df3
labels = mapper_labels(df['label'])

cubes = 20
overlap = 0.2
clusters = 5


mapper = km.KeplerMapper(verbose=2)

#projector = ensemble.IsolationForest(random_state=0, n_jobs=-1)
#projector.fit(df2)
#lens3 = mapper.fit_transform(df2, projection=PCA(n_components=1))

graph_new = mapper.map(
    df2,
    remove_duplicate_nodes=True,
    cover=km.Cover(n_cubes=cubes, perc_overlap=overlap),
    clusterer=cluster.KMeans(clusters))

my_colorscale = [[0.0, '#001219'],
             [0.1, '#005f73'],
             [0.2, '#0a9396'],
             [0.3, '#94d2bd'],
             [0.4, '#e9d8a6'],
             [0.5, '#ee9b00'],
             [0.6, '#ca6702'],
             [0.7, '#bb3e03'],
             [0.8, '#ae2012'],
             [0.9, '#9b2226'],
             [1.0, '#a50026']]  

mapper.visualize(
    graph_new,
    path_html="mappers/allInstances.html",
    title="'PLCO2AN', 'PLSO2AN', 'PLNOXAN', 'PLNGENAN',  'CarbonIntensity', 'Year', 'post2004RetrofitCosts', 'StateMineEmployment', 'CO2limitsOppose' (" +str(cubes)+ " cubes at " +str(overlap*100) + "% overlap) with Plant Labels",
    custom_tooltips=labels,
    color_values = colors,
    colorscale=my_colorscale,
    color_function_name=['PLCO2AN', 'PLSO2AN', 'PLNOXAN', 'CarbonIntensity', 'fuel'],
    node_color_function=['mean', 'median', 'max'],
    include_searchbar = True, 
    #include_intersection_selector= True
)



KeplerMapper(verbose=2)
Mapping on data shaped (3254, 5) using lens shaped (3254, 5)

Minimal points in hypercube before clustering: 2
Creating 3200000 hypercubes.
